In [ ]:
import json

def get_columns(schemas_file, ds_name):
    schema_text = spark.read.text(schemas_file, wholetext=True).first().value
    schemas = json.loads(schema_text)
    column_details = schemas[ds_name]
    columns = [col['column_name'] for col in sorted(column_details, key=lambda col: col['column_position'])]
    return columns

In [ ]:
ds_list = [
    'departments', 'categories', 'products',
    'customers', 'orders', 'order_items'
]

In [ ]:
base_dir = 'dbfs:/public/retail_db'

In [ ]:
tgt_base_dir = 'dbfs:/public/retail_db_parquet'

In [ ]:
columns = get_columns('dbfs:/public/retail_db/schemas.json', 'orders')

In [ ]:
orders = spark.read.csv('dbfs:/public/retail_db/orders', inferSchema=True).toDf(*columns)

In [ ]:
orders. \
    write. \
    mode('overwrite'). \
    parquet('dbfs:/public/retail_db_parquet/orders')

In [ ]:
%sql

SELECt * FROM PARQUET.`dbfs:/public/retail_db_parquet/orders`

In [ ]:
for ds in ds_list:
    print(f'Processing {ds} data')
    columns = get_columns(f'{base_dir}/schemas.json', ds)
    df = spark. \
        read. \
        csv(f'{base_dir}/{ds}', inferSchema=True, ). \
        toDf(*columns)
    df. \
        write. \
        mode('overwrite'). \
        parquet(f'{tgt_base_dir}/{ds}')